## Loick CUER et Léa FELDMAN

## Natural Language Processing : Information Retrieval Challenge

### Installation de BM25 et clonage du dossier de projet

In [ ]:
!pip install rank_bm25
!git clone https://github.com/cr-nlp/project1-2023.git

  Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)
fatal: destination path 'project1-2023' already exists and is not an empty directory.


### Importations des librairies principales

In [ ]:
import urllib.request as url_request
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score
from rank_bm25 import BM25Okapi

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Fonction qui charge le corpus

In [ ]:
def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc

## BM25 basique

In [ ]:
def text2TokenList(text):
	stopword = stopwords.words('english')
	#print("LEN DE STOPWORD=",len(stopword))
	word_tokens = word_tokenize(text.lower())
	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>2]
	return word_tokens_without_stops

def run_bm25_only(startDoc,endDoc):

	dicDoc, dicReq, dicReqDoc = loadNFCorpus()

	docsToKeep=[]
	reqsToKeep=[]
	dicReqDocToKeep=defaultdict(dict)

	#150
	ndcgTop=10
	#print("ndcgTop=",ndcgTop,"nbDocsToKeep=",nbDocsToKeep)

	i=startDoc
	for reqId in dicReqDoc:
		if i > (endDoc - startDoc) :  #nbDocsToKeep:
			break
		for docId in dicReqDoc[reqId]:
			dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
			docsToKeep.append(docId)
			i = i + 1
		reqsToKeep.append(reqId)
	docsToKeep = list(set(docsToKeep))

	#"""
	allVocab ={}
	for k in docsToKeep:
		docTokenList = text2TokenList(dicDoc[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListDoc = list(allVocab)
	#print("doc vocab=",len(allVocabListDoc))
	allVocab ={}
	for k in reqsToKeep:
		docTokenList = text2TokenList(dicReq[k])
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListReq = list(allVocab)

	from rank_bm25 import BM25Okapi

	corpusDocTokenList = []
	corpusReqTokenList = {}
	corpusDocName=[]
	corpusDicoDocName={}
	i = 0
	for k in docsToKeep:
		docTokenList = text2TokenList(dicDoc[k])
		corpusDocTokenList.append(docTokenList)
		corpusDocName.append(k)
		corpusDicoDocName[k] = i
		i = i + 1

	#print("reqs...")
	corpusReqName=[]
	corpusDicoReqName={}
	i = 0
	for k in reqsToKeep:
		reqTokenList = text2TokenList(dicReq[k])
		corpusReqTokenList[k] = reqTokenList
		corpusReqName.append(k)
		corpusDicoReqName[k] = i
		i = i + 1

	#print("bm25 doc indexing...")
	bm25 = BM25Okapi(corpusDocTokenList)

	ndcgCumul=0
	corpusReqVec={}
	ndcgBM25Cumul=0
	nbReq=0

	from sklearn.metrics import ndcg_score
	for req in corpusReqTokenList:
		j=0
		reqTokenList = corpusReqTokenList[req]
		doc_scores = bm25.get_scores(reqTokenList)
		trueDocs = np.zeros(len(corpusDocTokenList))

		for docId in corpusDicoDocName:
			if req in dicReqDocToKeep:
				if docId in dicReqDocToKeep[req]:
					#get position docId
					posDocId = corpusDicoDocName[docId]
					trueDocs[posDocId] = dicReqDocToKeep[req][docId]
					#print("TOKEEP=",docId)
					#print(trueDocs)
		ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
		nbReq = nbReq + 1
	ndcgBM25Cumul = ndcgBM25Cumul / nbReq
	return ndcgBM25Cumul

In [ ]:
print("Résultat en prenant seulement 150 documents en compte : "+str(run_bm25_only(0,150)))

Résultat en prenant seulement 150 documents en compte : 0.7519377590245511


### Pre-Processing alternatifs et BM25

La fonction qui se charge du pre-processing dans le code de base est text2TokenList. Apportons-y quelques changements.

In [ ]:
def text2TokenList(text, use_bigrams=True, use_trigrams=True):
    stopword = stopwords.words('english') #load the list of english stopwords
    text = re.sub(r'[^a-zA-Z]', ' ', text) # delete characters that are not letters
    text = text.lower() #convert words into lowercase
    word_tokens = word_tokenize(text) # tokenize our sentences into tokens (words)
    word_tokens_filtered = [word for word in word_tokens if word not in stopword and len(word) > 2] #filter stopwords and short words
    lemmatizer = WordNetLemmatizer() #lemmatize our tokens
    word_tokens_lemmatized = [lemmatizer.lemmatize(token) for token in word_tokens_filtered]
    if use_bigrams: #add bigrams
        bigrams = list(nltk.bigrams(word_tokens_filtered))
        word_tokens_lemmatized.extend([' '.join(bigram) for bigram in bigrams])
    if use_trigrams: #add trigrams
        trigrams = list(nltk.trigrams(word_tokens_filtered))
        word_tokens_lemmatized.extend([' '.join(trigram) for trigram in trigrams])
    return word_tokens_lemmatized

def run_bm25_only(startDoc,endDoc):

	dicDoc, dicReq, dicReqDoc = loadNFCorpus()

	docsToKeep=[]
	reqsToKeep=[]
	dicReqDocToKeep=defaultdict(dict)

	#150
	ndcgTop=10
	#print("ndcgTop=",ndcgTop,"nbDocsToKeep=",nbDocsToKeep)

	i=startDoc
	for reqId in dicReqDoc:
		if i > (endDoc - startDoc) :  #nbDocsToKeep:
			break
		for docId in dicReqDoc[reqId]:
			dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
			docsToKeep.append(docId)
			i = i + 1
		reqsToKeep.append(reqId)
	docsToKeep = list(set(docsToKeep))

	#"""
	allVocab ={}
	for k in docsToKeep:
		docTokenList = text2TokenList(dicDoc[k],True,True)
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListDoc = list(allVocab)
	#print("doc vocab=",len(allVocabListDoc))
	allVocab ={}
	for k in reqsToKeep:
		docTokenList = text2TokenList(dicReq[k],True,True)
		#print(docTokenList)
		for word in docTokenList:
			if word not in allVocab:
				allVocab[word] = word
	allVocabListReq = list(allVocab)

	from rank_bm25 import BM25Okapi

	corpusDocTokenList = []
	corpusReqTokenList = {}
	corpusDocName=[]
	corpusDicoDocName={}
	i = 0
	for k in docsToKeep:
		docTokenList = text2TokenList(dicDoc[k],True,True)
		corpusDocTokenList.append(docTokenList)
		corpusDocName.append(k)
		corpusDicoDocName[k] = i
		i = i + 1

	#print("reqs...")
	corpusReqName=[]
	corpusDicoReqName={}
	i = 0
	for k in reqsToKeep:
		reqTokenList = text2TokenList(dicReq[k],True,True)
		corpusReqTokenList[k] = reqTokenList
		corpusReqName.append(k)
		corpusDicoReqName[k] = i
		i = i + 1

	#print("bm25 doc indexing...")
	bm25 = BM25Okapi(corpusDocTokenList)

	ndcgCumul=0
	corpusReqVec={}
	ndcgBM25Cumul=0
	nbReq=0

	from sklearn.metrics import ndcg_score
	for req in corpusReqTokenList:
		j=0
		reqTokenList = corpusReqTokenList[req]
		doc_scores = bm25.get_scores(reqTokenList)
		trueDocs = np.zeros(len(corpusDocTokenList))

		for docId in corpusDicoDocName:
			if req in dicReqDocToKeep:
				if docId in dicReqDocToKeep[req]:
					#get position docId
					posDocId = corpusDicoDocName[docId]
					trueDocs[posDocId] = dicReqDocToKeep[req][docId]
					#print("TOKEEP=",docId)
					#print(trueDocs)
		ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
		nbReq = nbReq + 1
	ndcgBM25Cumul = ndcgBM25Cumul / nbReq
	return ndcgBM25Cumul


In [ ]:
print("Résultat en prenant seulement 150 documents en compte : "+str(run_bm25_only(0,150)))

Résultat en prenant seulement 150 documents en compte : 0.7612331097196332


In [ ]:
dicDoc, dicReq, dicReqDoc = loadNFCorpus()

first_doc_key = next(iter(dicDoc))  #Let's take a look at the first document of our corpus and the output of our pre-processing applicated on this document

print(f"Clé du premier document : {first_doc_key}")
print(f"Contenu du premier document : {first_doc_value}")
print(text2TokenList(first_doc_value, use_bigrams=True, use_trigrams=True))

Clé du premier document : MED-118
Contenu du premier document : alkylphenols human milk relations dietary habits central taiwan pubmed ncbi abstract aims study determine concentrations num nonylphenol np num octylphenol op num human milk samples examine related factors including mothers demographics dietary habits women consumed median amount cooking oil significantly higher op concentrations num ng/g consumed num ng/g num op concentration significantly consumption cooking oil beta num num fish oil capsules beta num num adjustment age body mass index bmi np concentration significantly consumption fish oil capsules beta num num processed fish products beta num num food pattern cooking oil processed meat products factor analysis strongly op concentration human milk num determinations aid suggesting foods consumption nursing mothers order protect infants np/op exposure num elsevier rights reserved 

['alkylphenols', 'human', 'milk', 'relation', 'dietary', 'habit', 'central', 'taiwan', 'pu

Nous avons choisi la lemmatisation car elle est en général plus précise même si plus lente et plus complexe.

### Creation du modèle word2vec

In [ ]:
pip install gensim

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [ ]:
dicDoc, dicReq, dicReqDoc = loadNFCorpus()

def text2TokenList(text, use_bigrams=True, use_trigrams=True):
    stopword = stopwords.words('english') #load the list of english stopwords
    text = re.sub(r'[^a-zA-Z]', ' ', text) # delete characters that are not letters
    text = text.lower() #convert words into lowercase
    word_tokens = word_tokenize(text) # tokenize our sentences into tokens (words)
    word_tokens_filtered = [word for word in word_tokens if word not in stopword and len(word) > 2] #filter stopwords and short words
    lemmatizer = WordNetLemmatizer() #lemmatize our tokens
    word_tokens_lemmatized = [lemmatizer.lemmatize(token) for token in word_tokens_filtered]
    if use_bigrams: #add bigrams
        bigrams = list(nltk.bigrams(word_tokens_filtered))
        word_tokens_lemmatized.extend([' '.join(bigram) for bigram in bigrams])
    if use_trigrams: #add trigrams
        trigrams = list(nltk.trigrams(word_tokens_filtered))
        word_tokens_lemmatized.extend([' '.join(trigram) for trigram in trigrams])
    return word_tokens_lemmatized

# tokenize our corpus sentences to words
tokenized_corpus = [text2TokenList(doc, use_bigrams=True, use_trigrams=True) for doc in list(dicDoc.values())]

In [ ]:
model = Word2Vec(tokenized_corpus, vector_size=400, window=8, min_count=3, workers=6, sg=0, epochs=100) #create our word2vec model

In [ ]:
bm25 = BM25Okapi(tokenized_corpus) #apply BM25 to our tokenized corpus

In [ ]:
def retrieve_documents_bm25(query, bm25, top_n=5):  #Tokenize the query, use BM25 model to compute relevance scores and return the top n documents with the best scores.
    query_tokens = text2TokenList(query)
    bm25_scores = bm25.get_scores(query_tokens)
    return np.argsort(-bm25_scores)[:top_n]

def evaluate_retrieval_bm25(dicReq, dicReqDoc, bm25, doc_ids, top_n=5): #Assesses the retrieval performance of a BM25 model by computing the mean Normalized Discounted Cumulative Gain (NDCG)
    ndcg_scores = []
    for req_id, query in dicReq.items():
        true_scores_dict = dicReqDoc[req_id]
        retrieved_doc_indices = retrieve_documents_bm25(query, bm25, top_n=top_n)
        retrieved_doc_ids = [doc_ids[idx] for idx in retrieved_doc_indices]
        retrieved_scores = np.array([true_scores_dict.get(doc_id, 0) for doc_id in retrieved_doc_ids], dtype=np.float64)
        ideal_scores_list = sorted(true_scores_dict.values(), reverse=True)[:len(retrieved_scores)]
        ideal_scores_list.extend([0] * (top_n - len(ideal_scores_list)))
        ideal_scores = np.array(ideal_scores_list, dtype=np.float64)
        ndcg = ndcg_score([ideal_scores], [retrieved_scores], k=top_n)
        ndcg_scores.append(ndcg)
    return np.mean(ndcg_scores)

doc_ids = list(dicDoc.keys())
bm25_ndcg = evaluate_retrieval_bm25(dicReq, dicReqDoc, bm25, doc_ids)
print("Résultat du modèle BM25 avec pre-processing et word2vec : "+str(bm25_ndcg))

Résultat du modèle BM25 avec pre-processing et word2vec : 0.9477037341451985
